In [20]:
import sys
import os
from TopoAnalysis import dem as d
import numpy as np
import matplotlib.pyplot as plt

In [1]:
import os
import sys

# Check if the path exists
fluvial_path = '/Users/Glong1/Desktop/Andes/AndesTG/FluvialLandformInversion-master/'
print(f"Path exists: {os.path.exists(fluvial_path)}")

# List what's inside
if os.path.exists(fluvial_path):
    print("\nContents:")
    print(os.listdir(fluvial_path))
    
    # Check for python subdirectory
    python_path = os.path.join(fluvial_path, 'python')
    if os.path.exists(python_path):
        print("\nPython directory contents:")
        print(os.listdir(python_path))

Path exists: True

Contents:
['.DS_Store', 'python', 'CLEANUP_SUMMARY.md', 'PROJECT_COMPLETE.txt', 'MISFIT_FORMULA_FIX.md', 'README.md', 'GIT_CLEANUP_INSTRUCTIONS.md', '.gitignore', 'PYTHON_PORT_SUMMARY.md', 'matlab', '.ipynb_checkpoints', 'DEM_INTEGRATION_SUMMARY.md']

Python directory contents:
['ARCHITECTURE.md', '.DS_Store', 'requirements.txt', 'tests', 'fluvial_inversion', 'README.md', 'README_DEM_INTEGRATION.md', 'setup.py', 'examples', '.ipynb_checkpoints', 'venv', 'README_VENV.md']


In [4]:
"""
Rapel River Fluvial Landform Inversion Workflow

This notebook processes the Rapel DEM and runs fluvial inversion to reconstruct
uplift rate history for sample locations.

Working from: AndesTG/Rapel/base_level/LGoren_Fluvial_landform_inversion.ipynb
"""

# ============================================================================
# SETUP AND IMPORTS
# ============================================================================

import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add FluvialLandformInversion to path
sys.path.insert(0, '/Users/Glong1/Desktop/Andes/AndesTG/FluvialLandformInversion-master/python')

# Import TopoAnalysis
from TopoAnalysis import dem as d

# Import fluvial inversion functions
from fluvial_inversion import (
    calculate_chi,
    invert_block_uplift,
    findm_slope_area,
    invert_with_different_gamma,
    calibrate_k_total_uplift,
    bootstrap_invert_block_uplift
)

print("✓ All packages imported successfully")

✓ All packages imported successfully


In [5]:
# ============================================================================
# STEP 1: LOAD AND PROCESS DEM
# ============================================================================

print("\n" + "="*70)
print("STEP 1: LOADING AND PROCESSING DEM")
print("="*70)

# Load the Rapel DEM
dem_path = '/Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver/rapel_SRTMGL130m_dem_utm.tif'
print(f"\nLoading DEM from: {dem_path}")

elevation = d.Elevation.load(dem_path)
print(f"✓ DEM loaded")
print(f"  Size: {elevation._georef_info.nx} x {elevation._georef_info.ny} pixels")
print(f"  Resolution: {elevation._georef_info.dx} m")
print(f"  Center coords: ({elevation._georef_info.xllcenter:.2f}, {elevation._georef_info.yllcenter:.2f})")

# Fill depressions
print("\nFilling depressions...")
filled = d.FilledElevation(elevation=elevation)
print("✓ Depressions filled")

# Calculate flow direction
print("\nCalculating D8 flow directions...")
flow_direction = d.FlowDirectionD8(flooded_dem=filled)
print("✓ Flow directions calculated")

# Calculate drainage area
print("\nCalculating drainage area...")
area = d.Area(flow_direction=flow_direction)
print("✓ Drainage area calculated")

# Optional: save processed data
save_processed = False  # Set to True if you want to save
if save_processed:
    filled.save('rapel_dem_utm_filled30m')
    flow_direction.save('rapel_fd_utm30m')
    area.save('rapel_area_utm30m')
    print("✓ Processed data saved")


STEP 1: LOADING AND PROCESSING DEM

Loading DEM from: /Users/Glong1/Desktop/Andes/Andes_watersheds/RapelRiver/rapel_SRTMGL130m_dem_utm.tif
✓ DEM loaded
  Size: 14877 x 12203 pixels
  Resolution: 27.350566245475594 m
  Center coords: (125032.27, 6046054.07)

Filling depressions...
✓ Depressions filled

Calculating D8 flow directions...
✓ Flow directions calculated

Calculating drainage area...
✓ Drainage area calculated


In [8]:
# ============================================================================
# STEP 2: DEFINE SAMPLE LOCATIONS
# ============================================================================

print("\n" + "="*70)
print("STEP 2: DEFINE SAMPLE LOCATIONS")
print("="*70)

# Define your sample points here
# Format: [['Sample_Name', easting, northing], ...]
# Replace these with your actual sample coordinates

samples = [
    # Example format - replace with your samples:
    ['YOUR_SAMPLE_1', 240000.0, 6244000.0],
    ['RP-S2', 246249.2076, 6240448.447],
    ['RP-S3',250212.9354, 6237948.848],
    ['RP-S3u',250603.5211, 6228481.157],
    ['RP-S4u',262281.9748, 6217862.614]
]

print(f"\nNumber of samples defined: {len(samples)}")
if len(samples) > 0:
    print("\nSample locations:")
    for sample_name, x, y in samples:
        print(f"  {sample_name}: ({x:.1f}, {y:.1f})")
else:
    print("\n⚠ WARNING: No samples defined!")
    print("Please add your sample coordinates in the 'samples' list above.")
    print("Format: ['SAMPLE_NAME', easting, northing]")


STEP 2: DEFINE SAMPLE LOCATIONS

Number of samples defined: 5

Sample locations:
  YOUR_SAMPLE_1: (240000.0, 6244000.0)
  RP-S2: (246249.2, 6240448.4)
  RP-S3: (250212.9, 6237948.8)
  RP-S3u: (250603.5, 6228481.2)
  RP-S4u: (262282.0, 6217862.6)


In [9]:
# ============================================================================
# STEP 3: EXTRACT DRAINAGE BASINS FOR EACH SAMPLE
# ============================================================================

print("\n" + "="*70)
print("STEP 3: EXTRACTING DRAINAGE BASINS")
print("="*70)

# Dictionary to store basin data for each sample
basin_data = {}

for sample_name, x, y in samples:
    print(f"\nProcessing {sample_name}...")
    
    try:
        # Create basin mask
        basin_mask = d.Mask(flow_direction=flow_direction, outlets=((x, y),))
        
        # Calculate basin area
        total_area = np.sum(basin_mask._griddata) * np.power(basin_mask._georef_info.dx, 2)
        total_area_km2 = total_area / 1.0E6
        
        print(f"  ✓ Basin area: {total_area_km2:.2f} km²")
        
        # Store basin mask for later use
        basin_data[sample_name] = {
            'mask': basin_mask,
            'outlet_x': x,
            'outlet_y': y,
            'area_km2': total_area_km2
        }
        
    except Exception as e:
        print(f"  ✗ Failed: {e}")

print(f"\n✓ Successfully extracted {len(basin_data)} basins")


STEP 3: EXTRACTING DRAINAGE BASINS

Processing YOUR_SAMPLE_1...
  ✓ Basin area: 0.32 km²

Processing RP-S2...
  ✓ Basin area: 33.99 km²

Processing RP-S3...
  ✓ Basin area: 209.83 km²

Processing RP-S3u...
  ✓ Basin area: 112.17 km²

Processing RP-S4u...
  ✓ Basin area: 9.35 km²

✓ Successfully extracted 5 basins


In [17]:
# ============================================================================
# STEP 4: EXTRACT FLUVIAL NETWORK DATA FOR INVERSION
# ============================================================================

print("\n" + "="*70)
print("STEP 4: EXTRACTING FLUVIAL NETWORK DATA")
print("="*70)

def extract_network_data(elevation, area, flow_direction, basin_mask, min_drainage_area=1e6):
    """
    Extract network data from TopoAnalysis objects for fluvial inversion.
    
    Parameters
    ----------
    elevation : Elevation object
        DEM elevation data
    area : Area object
        Drainage area grid
    flow_direction : FlowDirectionD8 object
        Flow direction grid
    basin_mask : Mask object
        Basin mask
    min_drainage_area : float
        Minimum drainage area threshold [m²]
    
    Returns
    -------
    dict : Network data for inversion
    """
    
    # Get grid data
    z_grid = elevation._griddata
    area_grid = area._griddata
    fd_grid = flow_direction._griddata
    mask_grid = basin_mask._griddata
    
    # Get georeferencing info (dx is used for both x and y since pixels are square)
    dx = elevation._georef_info.dx
    xll = elevation._georef_info.xllcenter
    yll = elevation._georef_info.yllcenter
    
    # Apply basin mask and area threshold
    valid = (mask_grid > 0) & (area_grid >= min_drainage_area) & (z_grid > 0)
    
    # Get indices of valid pixels
    rows, cols = np.where(valid)
    n_pixels = len(rows)
    
    # Extract coordinates (using dx for both dimensions since pixels are square)
    x = xll + cols * dx
    y = yll + rows * dx
    z = z_grid[rows, cols]
    area_array = area_grid[rows, cols]
    
    # Build receiver array (convert flow direction to receiver index)
    # Flow direction codes: 1=E, 2=SE, 4=S, 8=SW, 16=W, 32=NW, 64=N, 128=NE
    flow_dir_map = {
        1: (0, 1),    # E
        2: (1, 1),    # SE
        4: (1, 0),    # S
        8: (1, -1),   # SW
        16: (0, -1),  # W
        32: (-1, -1), # NW
        64: (-1, 0),  # N
        128: (-1, 1)  # NE
    }
    
    # Create mapping from (row, col) to index
    coord_to_idx = {(r, c): i for i, (r, c) in enumerate(zip(rows, cols))}
    
    # Build receiver array
    rec_array = np.full(n_pixels, -1, dtype=int)
    for i, (r, c) in enumerate(zip(rows, cols)):
        fd_val = fd_grid[r, c]
        if fd_val in flow_dir_map:
            dr, dc = flow_dir_map[fd_val]
            rec_r, rec_c = r + dr, c + dc
            if (rec_r, rec_c) in coord_to_idx:
                rec_array[i] = coord_to_idx[(rec_r, rec_c)]
    
    # Calculate slope (simple gradient)
    slope_array = np.zeros(n_pixels)
    for i, (r, c) in enumerate(zip(rows, cols)):
        rec_idx = rec_array[i]
        if rec_idx >= 0:
            dz = z[i] - z[rec_idx]
            dist = np.sqrt((x[i] - x[rec_idx])**2 + (y[i] - y[rec_idx])**2)
            if dist > 0:
                slope_array[i] = max(dz / dist, 1e-6)  # Avoid zero/negative slopes
    
    return {
        'x': x,
        'y': y,
        'z': z,
        'area_array': area_array,
        'rec_array': rec_array,
        'slope_array': slope_array,
        'n_pixels': n_pixels
    }

# Extract network data for each basin
network_data = {}

min_drainage_area = 1e6  # 1 km² threshold - adjust as needed

for sample_name, basin_info in basin_data.items():
    print(f"\nExtracting network for {sample_name}...")
    
    try:
        net_data = extract_network_data(
            filled,
            area,
            flow_direction,
            basin_info['mask'],
            min_drainage_area=min_drainage_area
        )
        
        network_data[sample_name] = net_data
        print(f"  ✓ Extracted {net_data['n_pixels']} pixels")
        print(f"    Elevation range: [{net_data['z'].min():.1f}, {net_data['z'].max():.1f}] m")
        print(f"    Area range: [{net_data['area_array'].min():.2e}, {net_data['area_array'].max():.2e}] m²")
        
    except Exception as e:
        print(f"  ✗ Failed: {e}")


STEP 4: EXTRACTING FLUVIAL NETWORK DATA

Extracting network for YOUR_SAMPLE_1...
  ✓ Extracted 0 pixels
  ✗ Failed: zero-size array to reduction operation minimum which has no identity

Extracting network for RP-S2...
  ✓ Extracted 980 pixels
    Elevation range: [21.0, 195.0] m
    Area range: [1.00e+06, 3.40e+07] m²

Extracting network for RP-S3...
  ✓ Extracted 4986 pixels
    Elevation range: [23.0, 337.0] m
    Area range: [1.00e+06, 2.10e+08] m²

Extracting network for RP-S3u...
  ✓ Extracted 2521 pixels
    Elevation range: [213.0, 327.0] m
    Area range: [1.00e+06, 1.12e+08] m²

Extracting network for RP-S4u...
  ✓ Extracted 233 pixels
    Elevation range: [166.0, 281.0] m
    Area range: [1.01e+06, 9.35e+06] m²


In [18]:
# ============================================================================
# STEP 5: CALCULATE CONCAVITY INDEX (m) FOR EACH BASIN
# ============================================================================

print("\n" + "="*70)
print("STEP 5: CALCULATING CONCAVITY INDEX")
print("="*70)

m_values = {}

for sample_name, net_data in network_data.items():
    print(f"\nCalculating m for {sample_name}...")
    
    try:
        m, m_lb, m_ub, R2 = findm_slope_area(
            net_data['slope_array'],
            net_data['area_array'],
            to_plot=False
        )
        
        m_values[sample_name] = m
        print(f"  ✓ m = {m:.4f} (95% CI: [{m_lb:.4f}, {m_ub:.4f}]), R² = {R2:.4f}")
        
    except Exception as e:
        print(f"  ✗ Failed: {e}")
        m_values[sample_name] = 0.45  # Default value


STEP 5: CALCULATING CONCAVITY INDEX

Calculating m for YOUR_SAMPLE_1...
  ✗ Failed: Need at least 2 positive slope values for regression

Calculating m for RP-S2...
  ✓ m = 1.3357 (95% CI: [1.6650, 1.0063]), R² = 0.0609

Calculating m for RP-S3...
  ✓ m = 0.6430 (95% CI: [0.7241, 0.5620]), R² = 0.0463

Calculating m for RP-S3u...
  ✓ m = 0.7362 (95% CI: [0.8580, 0.6145]), R² = 0.0529

Calculating m for RP-S4u...
  ✓ m = 0.4901 (95% CI: [1.5008, -0.5206]), R² = 0.0040


In [19]:
# ============================================================================
# STEP 6: CALCULATE CHI COORDINATE
# ============================================================================

print("\n" + "="*70)
print("STEP 6: CALCULATING CHI COORDINATE")
print("="*70)

chi_data = {}

for sample_name, net_data in network_data.items():
    if sample_name not in m_values:
        continue
        
    print(f"\nCalculating chi for {sample_name}...")
    
    try:
        chi = calculate_chi(
            net_data['x'],
            net_data['y'],
            net_data['rec_array'],
            net_data['area_array'],
            m=m_values[sample_name],
            A0=1.0
        )
        
        chi_data[sample_name] = chi
        print(f"  ✓ Chi range: [{chi.min():.2f}, {chi.max():.2f}] m")
        
    except Exception as e:
        print(f"  ✗ Failed: {e}")




STEP 6: CALCULATING CHI COORDINATE

Calculating chi for YOUR_SAMPLE_1...
  ✗ Failed: zero-size array to reduction operation minimum which has no identity

Calculating chi for RP-S2...
  ✗ Failed: m should typically be between 0 and 1

Calculating chi for RP-S3...
  ✗ Failed: Receiver -1 for pixel 0 not found

Calculating chi for RP-S3u...
  ✗ Failed: Receiver -1 for pixel 0 not found

Calculating chi for RP-S4u...
  ✗ Failed: Receiver -1 for pixel 0 not found


In [13]:
# ============================================================================
# STEP 7: RUN FLUVIAL INVERSION
# ============================================================================

print("\n" + "="*70)
print("STEP 7: RUNNING FLUVIAL INVERSION")
print("="*70)

# Inversion parameters
gamma = 1.0  # Regularization parameter
q = 5        # Number of time intervals

print(f"\nInversion parameters:")
print(f"  Gamma (regularization): {gamma}")
print(f"  q (time intervals): {q}")

inversion_results = {}

for sample_name in network_data.keys():
    if sample_name not in chi_data:
        continue
        
    print(f"\n{'='*50}")
    print(f"Inverting {sample_name}")
    print('='*50)
    
    try:
        # Run inversion
        Ustar, tstar, misfit = invert_block_uplift(
            chi=chi_data[sample_name],
            z=network_data[sample_name]['z'],
            gamma=gamma,
            q=q,
            to_plot=False
        )
        
        inversion_results[sample_name] = {
            'Ustar': Ustar,
            'tstar': tstar,
            'misfit': misfit,
            'm': m_values[sample_name],
            'chi': chi_data[sample_name]
        }
        
        print(f"\n✓ Inversion complete")
        print(f"  RMS misfit: {misfit:.2f} m")
        print(f"\n  Non-dimensional uplift rates (U*):")
        for i, u in enumerate(Ustar):
            print(f"    Interval {i+1}: {u:.6f}")
            
    except Exception as e:
        print(f"✗ Inversion failed: {e}")




STEP 7: RUNNING FLUVIAL INVERSION

Inversion parameters:
  Gamma (regularization): 1.0
  q (time intervals): 5


In [14]:
# ============================================================================
# STEP 8: CALIBRATE TO DIMENSIONAL UNITS (OPTIONAL)
# ============================================================================

print("\n" + "="*70)
print("STEP 8: CALIBRATION TO DIMENSIONAL UNITS")
print("="*70)

# Define calibration constraints
# Adjust these values based on your knowledge of the region
H_total = 500.0  # Total uplift [m] - CHANGE THIS
t_H = 2e6        # Time period [years] - CHANGE THIS
A0 = 1.0         # Reference drainage area

print(f"\nCalibration constraint: {H_total} m uplift over {t_H/1e6:.1f} Ma")
print("\n⚠ Adjust H_total and t_H values above based on your constraints!")

calibrated_results = {}

for sample_name, inv_result in inversion_results.items():
    print(f"\nCalibrating {sample_name}...")
    
    try:
        K, U, t = calibrate_k_total_uplift(
            H_total,
            t_H,
            inv_result['Ustar'],
            inv_result['tstar'],
            A0,
            inv_result['m'],
            to_plot=False
        )
        
        calibrated_results[sample_name] = {
            'K': K,
            'U': U,
            't': t
        }
        
        print(f"  ✓ K = {K:.4e} m^(1-2m)/yr")
        print(f"\n  Dimensional uplift rates [mm/yr]:")
        for i, u_dim in enumerate(U):
            t_start = t[i] / 1e6
            t_end = t[i+1] / 1e6
            print(f"    {t_start:.2f} - {t_end:.2f} Ma: {u_dim*1e3:.4f} mm/yr")
            
    except Exception as e:
        print(f"  ✗ Failed: {e}")




STEP 8: CALIBRATION TO DIMENSIONAL UNITS

Calibration constraint: 500.0 m uplift over 2.0 Ma

⚠ Adjust H_total and t_H values above based on your constraints!


In [15]:
# ============================================================================
# STEP 9: VISUALIZE RESULTS
# ============================================================================

print("\n" + "="*70)
print("STEP 9: CREATING VISUALIZATIONS")
print("="*70)

def plot_sample_results(sample_name, network, chi, inversion, calibrated=None):
    """Create diagnostic plots for a single sample."""
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'Fluvial Inversion Results: {sample_name}', fontsize=14, fontweight='bold')
    
    # Plot 1: Chi-elevation profile
    ax = axes[0, 0]
    ax.scatter(chi, network['z'], c='blue', alpha=0.5, s=5)
    ax.set_xlabel('χ [m]', fontsize=11)
    ax.set_ylabel('Elevation [m]', fontsize=11)
    ax.set_title('Chi-Elevation Profile', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Slope-area relationship
    ax = axes[0, 1]
    valid = (network['slope_array'] > 0) & (network['area_array'] > 0)
    ax.loglog(network['area_array'][valid], network['slope_array'][valid], 
              'o', alpha=0.3, markersize=3, color='red')
    ax.set_xlabel('Drainage Area [m²]', fontsize=11)
    ax.set_ylabel('Slope', fontsize=11)
    ax.set_title(f"Slope-Area (m = {inversion['m']:.4f})", fontsize=12)
    ax.grid(True, alpha=0.3, which='both')
    
    # Plot 3: Non-dimensional uplift history
    ax = axes[0, 2]
    Ustar = inversion['Ustar']
    tstar = inversion['tstar']
    t_plot = []
    U_plot = []
    for i in range(len(Ustar)):
        t_plot.extend([tstar[i], tstar[i+1]])
        U_plot.extend([Ustar[i], Ustar[i]])
    ax.plot(t_plot, U_plot, 'b-', linewidth=2)
    ax.set_xlabel('t* [m]', fontsize=11)
    ax.set_ylabel('U*', fontsize=11)
    ax.set_title(f"Non-dim Uplift (misfit={inversion['misfit']:.2f}m)", fontsize=12)
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Drainage area histogram
    ax = axes[1, 0]
    ax.hist(np.log10(network['area_array']), bins=30, alpha=0.7, 
            edgecolor='black', color='green')
    ax.set_xlabel('log₁₀(Drainage Area) [m²]', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title('Drainage Area Distribution', fontsize=12)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Plot 5: Dimensional uplift history (if calibrated)
    ax = axes[1, 1]
    if calibrated is not None:
        U = calibrated['U']
        t = calibrated['t']
        t_plot_dim = []
        U_plot_dim = []
        for i in range(len(U)):
            t_plot_dim.extend([t[i]/1e6, t[i+1]/1e6])
            U_plot_dim.extend([U[i]*1e3, U[i]*1e3])
        ax.plot(t_plot_dim, U_plot_dim, 'g-', linewidth=2)
        ax.set_xlabel('Time [Ma]', fontsize=11)
        ax.set_ylabel('Uplift Rate [mm/yr]', fontsize=11)
        ax.set_title(f"Dimensional Uplift (K={calibrated['K']:.2e})", fontsize=12)
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'No calibration\navailable', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('Dimensional Uplift', fontsize=12)
    
    # Plot 6: Elevation histogram
    ax = axes[1, 2]
    ax.hist(network['z'], bins=30, alpha=0.7, edgecolor='black', color='purple')
    ax.set_xlabel('Elevation [m]', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title('Elevation Distribution', fontsize=12)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    return fig

# Create plots for each sample
for sample_name in inversion_results.keys():
    print(f"\nCreating plots for {sample_name}...")
    
    try:
        fig = plot_sample_results(
            sample_name,
            network_data[sample_name],
            chi_data[sample_name],
            inversion_results[sample_name],
            calibrated_results.get(sample_name)
        )
        
        # Save figure
        filename = f'{sample_name}_inversion_results.png'
        fig.savefig(filename, dpi=150, bbox_inches='tight')
        print(f"  ✓ Saved to {filename}")
        
        plt.show()
        
    except Exception as e:
        print(f"  ✗ Failed to create plot: {e}")




STEP 9: CREATING VISUALIZATIONS


In [16]:
# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*70)
print("WORKFLOW COMPLETE!")
print("="*70)

print(f"\nProcessed {len(basin_data)} basins:")
for sample_name in basin_data.keys():
    print(f"\n{sample_name}:")
    print(f"  Basin area: {basin_data[sample_name]['area_km2']:.2f} km²")
    
    if sample_name in network_data:
        print(f"  Network pixels: {network_data[sample_name]['n_pixels']}")
    
    if sample_name in m_values:
        print(f"  Concavity (m): {m_values[sample_name]:.4f}")
    
    if sample_name in inversion_results:
        print(f"  Inversion misfit: {inversion_results[sample_name]['misfit']:.2f} m")
        
    if sample_name in calibrated_results:
        U_mean = np.mean(calibrated_results[sample_name]['U']) * 1e3
        print(f"  Mean uplift rate: {U_mean:.4f} mm/yr")

print("\n" + "="*70)


WORKFLOW COMPLETE!

Processed 5 basins:

YOUR_SAMPLE_1:
  Basin area: 0.32 km²

RP-S2:
  Basin area: 33.99 km²

RP-S3:
  Basin area: 209.83 km²

RP-S3u:
  Basin area: 112.17 km²

RP-S4u:
  Basin area: 9.35 km²

